<a href="https://colab.research.google.com/github/triciat/ML100-Days/blob/master/Day_018_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測

In [4]:
import requests

url = 'http://ai100.cupoy.com/file-download/part02/titanic_train.csv'
r = requests.get(url, allow_redirects=True)
open('titanic_train.csv', 'wb').write(r.content)


url = 'http://ai100.cupoy.com/file-download/part02/titanic_test.csv'
r = requests.get(url, allow_redirects=True)
open('titanic_test.csv', 'wb').write(r.content)

28210

In [6]:
# 載入套件與資料
import pandas as pd
import numpy as np

data_path = './'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')
df_train.shape

(891, 12)

In [7]:
# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
# 秀出資料欄位的類型與數量
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df = dtype_df.groupby("Column Type").aggregate('count').reset_index()
dtype_df

,Column Type,Count
0,int64,3
1,float64,2
2,object,5


In [9]:
#確定只有 int64, float64, object 三種類型後, 分別將欄位名稱存於三個 list 中
int_features = []
float_features = []
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64':
        float_features.append(feature)
    elif dtype == 'int64':
        int_features.append(feature)
    else:
        object_features.append(feature)
print(f'{len(int_features)} Integer Features : {int_features}\n')
print(f'{len(float_features)} Float Features : {float_features}\n')
print(f'{len(object_features)} Object Features : {object_features}')

3 Integer Features : ['Pclass', 'SibSp', 'Parch']

2 Float Features : ['Age', 'Fare']

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


# 作業1 
* 試著執行作業程式，觀察三種類型 (int / float / object) 的欄位分別進行( 平均 mean / 最大值 Max / 相異值 nunique )  
中的九次操作會有那些問題? 並試著解釋那些發生Error的程式區塊的原因?  

In [10]:
feature_labels = ['Int', 'Float', 'Object']
features = [int_features, float_features, object_features]
for label, feature in zip(feature_labels, features):
    print(f'=== {label} ===')
    print(f'features: {feature}')
    
    print('\n--- mean ---')
    print(df[feature].mean())
    print('\n--- max ---')
    print(df[feature].max())
    print('\n--- nunique ---')
    print(df[feature].nunique())
    print('\n')

=== Int ===
features: ['Pclass', 'SibSp', 'Parch']

--- mean ---
Pclass    2.294882
SibSp     0.498854
Parch     0.385027
dtype: float64

--- max ---
Pclass    3
SibSp     8
Parch     9
dtype: int64

--- nunique ---
Pclass    3
SibSp     7
Parch     8
dtype: int64


=== Float ===
features: ['Age', 'Fare']

--- mean ---
Age     29.881138
Fare    33.295479
dtype: float64

--- max ---
Age      80.0000
Fare    512.3292
dtype: float64

--- nunique ---
Age      98
Fare    281
dtype: int64


=== Object ===
features: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

--- mean ---
Series([], dtype: float64)

--- max ---
Name      van Melkebeke, Mr. Philemon
Sex                              male
Ticket                      WE/P 5735
dtype: object

--- nunique ---
Name        1307
Sex            2
Ticket       929
Cabin        186
Embarked       3
dtype: int64




# 作業2
* 思考一下，試著舉出今天五種類型以外的一種或多種資料類型，你舉出的新類型是否可以歸在三大類中的某些大類?  
所以三大類特徵中，哪一大類處理起來應該最複雜?

### 答案

圖片、影片，應該可以被歸在類別型。我認為將多媒體資料轉成資料特徵應該頗複雜,，另外還有地理資訊(GIS)的資料也算是multi-dimensional 的複雜性較高的資料。